# SANS data reduction - DMSC Summer School

In [ ]:
import numpy as np
import scipp as sc
import plopp as pp

## Process run with sample

In [ ]:
fname = 'SANS_without_sample_NeXus_1_pulse/mccode.h5'

In [ ]:
def load_nexus(fname):
    import scippnexus.v2 as sx
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        dg = f[...]
    events = sc.collapse(dg['entry1']['data']['detector_signal_event_dat'].data, keep='dim_0')
    params = dg['entry1']['simulation']['Param']
    columns = ['p', 'x', 'y', 'n', 'id', 't']
    events = {c: v.copy() for c, v in zip(columns, events.values())}
    weights = events.pop('p')
    weights.unit = 'counts'
    da = sc.DataArray(data=weights, coords=events)

    da.coords['y'].unit = 'm'
    da.coords['y'] += 0.25 * sc.units.m
    da.coords['x'].unit = 'm'
    z = sc.full_like(da.coords['y'], float(params['detector_distance']))
    da.coords['position'] = sc.spatial.as_vectors(da.coords['x'], da.coords['y'], z)
    da.coords['tof'] = da.coords.pop('t')
    da.coords['tof'].unit = 's'
    da.coords['tof'] = da.coords['tof'].to(unit='ms')
    
    da.coords['sample_position'] = sc.vector([0., 0., 0.], unit='m')
    da.coords['source_position'] = sc.vector([0., 0., -float(params['sample_distance'])], unit='m')
    return da

In [ ]:
# def load(fname):
#     import pandas as pd
#     df = pd.read_csv(fname, sep=' ', names=['p', 'x', 'y', 'n', 'id', 't'], comment='#', index_col=False)
#     ds = sc.compat.from_pandas(df)
#     weights = ds['p'].data
#     weights.unit = 'counts'
#     da = sc.DataArray(data=weights,
#                      coords={k: v.data for k, v in ds.items()})
#     da.coords['y'].unit = 'm'
#     da.coords['y'] += 0.25 * sc.units.m
#     y = da.coords['y']
#     x = sc.zeros_like(y)
#     z = sc.full_like(y, 3.)
#     da.coords['position'] = sc.spatial.as_vectors(x, y, z)
#     da.coords['tof'] = da.coords.pop('t')
#     da.coords['tof'].unit = 's'
#     da.coords['tof'] = da.coords['tof'].to(unit='ms')
    
#     da.coords['sample_position'] = sc.vector([0., 0., 0.], unit='m')
#     da.coords['source_position'] = sc.vector([0., 0., -150.], unit='m')
    
#     return da

In [ ]:
sample = load_nexus(fname)

In [ ]:
sample.hist(tof=200, y=200).plot(norm='log', vmin=1.0e-5)

In [ ]:
sample.hist(y=200).plot(norm='log')

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import elastic

graph = {**beamline(scatter=True), **elastic("tof")}
sc.show_graph(graph, simplified=True)

In [ ]:
sample.transform_coords("wavelength", graph=graph).hist(wavelength=200).plot()

In [ ]:
sample_q = sample.transform_coords("Q", graph=graph)
sample_q

In [ ]:
sample_h = sample_q.hist(Q=200)
sample_h.plot(norm='log', vmin=1.0e-3)

## Process background run

In [ ]:
fname = 'SANS_without_sample/detector_signal_event.dat'

background = load(fname)

In [ ]:
background.hist(y=200).plot()

In [ ]:
background_q = background.transform_coords("Q", graph=graph)

In [ ]:
background_h = background_q.hist(Q=200)
background_h.plot()

## Subtract background

In [ ]:
subbed = sample_h - background_h

In [ ]:
qbins = sc.linspace('Q', 5.0e-3, 0.2, 201, unit='1/angstrom')
qbins

In [ ]:
subbed = sample_q.hist(Q=qbins) - background_q.hist(Q=qbins)
subbed

In [ ]:
subbed.plot(norm='log', vmin=1.0e-3)

In [ ]:
%matplotlib widget
pp.plot({'raw': sample_q.hist(Q=qbins), 'background-subtracted': subbed}, norm='log', vmin=1.0e-3)